<a target="_blank" href="https://colab.research.google.com/github/rajshah4/LLM-Evaluation/blob/main/ragas_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Welcome to the ragas quickstart. We're going to get you up and running with ragas as qickly as you can so that you can go back to improving your Retrieval Augmented Generation pipelines while this library makes sure your changes are improving your entire pipeline.

to kick things of lets start with the data

In [1]:
%pip -q install ragas pandas datasets

Ragas also uses OpenAI for running some metrics so make sure you have your openai key ready and available in your environment

In [2]:
import os
import getpass

open_ai_key = getpass.getpass('Enter your OPENAI API Key')
os.environ['OPENAI_API_KEY'] = open_ai_key

Enter your OPENAI API Key··········


## The Data

Ragas performs a `ground_truth` free evaluation of your RAG pipelines. This is because for most people building a gold labeled dataset which represents in the distribution they get in production is a very expensive process.

Hence to work with ragas all you need are the following data
- question: `list[str]` - These are the questions you RAG pipeline will be evaluated on.
- answer: `list[str]` - The answer generated from the RAG pipeline and give to the user.
- contexts: `list[list[str]]` - The contexts which where passed into the LLM to answer the question.

Ideally your list of questions should reflect the questions your users give, including those that you have been problamatic in the past.

Here we're using an example dataset from on of the baselines we created for the [Financial Opinion Mining and Question Answering (fiqa) Dataset](https://sites.google.com/view/fiqa/) we created. If you want to want to know more about the baseline, feel free to check the `experiements/baseline` section

In [3]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

## Metrics

Ragas measures your pipeline's performance against two dimensions

1. Faithfulness: measures the factual consistency of the generated answer against the given context.
2. Relevancy: measures how relevant retrieved contexts and the generated answer are to the question.

Through repeated experiments, we have found that the quality of a RAG pipeline is highly dependent on these two dimensions. The final `ragas_score` is the harmonic mean of these two factors.

now lets import these metrics and understand more about what they denote

In [4]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

here you can see that we are using 4 metrics, but what do the represent?

1. answer_relevancy - a measure of how relevant the answer is to the question

2. faithfulness - the factual consistency of the answer to the context base on the question.

3. context_recall: measures the ability of the retriever to retrieve all the necessary information needed to answer the question.

4. context_precision - a measure of how relevant the retrieved context is to the question. Conveys quality of the retrieval pipeline.

**Note:** *by default these metrics are using OpenAI's API to compute the score. If you using this metric make sure you set the environment key `OPENAI_API_KEY` with your API key. You can also try other LLMs for evaluation, check the [llm guide](./guides/llms.ipynb) to learn more*

If you're interested in learning more, feel free to check the [docs](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

## Evaluation

Running the evalutation is as simple as calling evaluate on the `Dataset` with the metrics of your choice.

In [5]:
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(1)),
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ],
)

result

evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:03<00:00,  3.79s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:01<00:00,  1.21s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:04<00:00,  4.12s/it]


{'ragas_score': 0.2931, 'context_precision': 0.4706, 'faithfulness': 0.6667, 'answer_relevancy': 0.9769, 'context_recall': 0.1111}

and there you have the it, all the scores you need. `ragas_score` gives you a single metric that you can use while the other onces measure the different parts of your pipeline.

now if we want to dig into the results and figure out examples where your pipeline performed worse or really good you can easily convert it into a pandas array and use your standard analytics tools too!

In [6]:
df = result.to_pandas()
df.head()

,question,contexts,answer,ground_truths,context_precision,faithfulness,answer_relevancy,context_recall
0,How to deposit a cheque issued to an associate...,[Just have the associate sign the back and the...,\nThe best way to deposit a cheque issued to a...,[Have the check reissued to the proper payee.J...,0.470588,0.666667,0.976913,0.111111


And thats it!

You can check out the [ragas in action] notebook to get a feel of what is like to use it while trying to improve your pipelines.

if you have any suggestion/feedbacks/things your not happy about, please do share it in the [issue section](https://github.com/explodinggradients/ragas/issues). We love hearing from you 😁